<a href="https://colab.research.google.com/github/balalinux/Hello-World/blob/master/Unsloth_Train_TinyLlma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
ma, mi = torch.cuda.get_device_capability()

In [3]:
print(ma,mi)

7 5


In [4]:
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-rws1nnx1/unsloth_c43422a0cd4346b788505a14ea304636
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-rws1nnx1/unsloth_c43422a0cd4346b788505a14ea304636
  Resolved https://github.com/unslothai/unsloth.git to commit a030e802030d4619ba247d45c5819fb59e9addb3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 101.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [6]:
!pip install "git+https://github.com/huggingface/transformers.git"

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-lc5jy7zs
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-lc5jy7zs
  Resolved https://github.com/huggingface/transformers.git to commit 354775bc5755c4a6c47e008d28f27f8ccdcf8f8f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.38.0.dev0-py3-none-any.whl size=8487256 sha256=92786e7ea0e339c5cddb9b7027014b62f3692ee8d2a206c565d9669ada3316db
  Stored in directory: /tmp/pip-ephem-wheel-cache-kr1_lo5k/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2


In [8]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(

    model_name = "unsloth/tinyllama-bnb-4bit",

    max_seq_length = max_seq_length,

    dtype = dtype,

    load_in_4bit = load_in_4bit,

)

/usr/local/lib/python3.10/dist-packages/unsloth/__init__.py:66: UserWarning: Running `ldconfig /usr/lib64-nvidia` to link CUDA.
  warnings.warn(


config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.2
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth: unsloth/tinyllama-bnb-4bit can only handle sequence lengths of at most 2048.
But with kaiokendev's RoPE scaling of 2.0, it can be magically be extended to 4096!
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [9]:
from datasets import load_dataset



alpaca_prompt = """Below is an instruction that describes a task



### Instruction:

{}



### Input:

{}



### Response:

{}

"""



def formatting_prompts_func(examples):

    instructions = examples["instruction"]

    inputs = examples["input"]

    outputs = examples["output"]

    texts = []



    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output)

        texts.append(text)



    return {"text": texts}



dataset = load_dataset('yahma/alpaca-cleaned', split='train[:1%]')

dataset = dataset.map(formatting_prompts_func, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/518 [00:00<?, ? examples/s]

In [17]:
from trl import SFTTrainer

from transformers import TrainingArguments

from transformers.utils import logging

logging.set_verbosity_info()



model = FastLanguageModel.get_peft_model(

    model,

    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128

    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",

                      "gate_proj", "up_proj", "down_proj",],

    lora_alpha = 32,

    lora_dropout = 0,

    bias = "none",

    use_gradient_checkpointing = False,

    random_state = 3407,

    max_seq_length = max_seq_length,

)



trainer = SFTTrainer(

    model = model,

    train_dataset = dataset,

    dataset_text_field = "text",

    max_seq_length = max_seq_length,

    packing = True,

    args = TrainingArguments(

        per_device_train_batch_size = 2,

        gradient_accumulation_steps = 4,

        warmup_ratio = 0.1,

        num_train_epochs = 1,

        learning_rate = 2e-5,

        fp16 = not torch.cuda.is_bf16_supported(),

        bf16 = torch.cuda.is_bf16_supported(),

        logging_steps = 1,

        optim = "adamw_8bit",

        weight_decay = 0.1,

        lr_scheduler_type = "linear",

        seed = 3407,

        output_dir = "outputs",

    ),

)




Unsloth 2024.2 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading file tokenizer.model from cache at /root/.cache/huggingface/hub/models--unsloth--tinyllama-bnb-4bit/snapshots/fc56510003ea9d49362400b8a362345150802c31/tokenizer.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--unsloth--tinyllama-bnb-4bit/snapshots/fc56510003ea9d49362400b8a362345150802c31/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--unsloth--tinyllama-bnb-4bit/snapshots/fc56510003ea9d49362400b8a362345150802c31/special_tokens_map.json
loading file tokenizer_config.j

In [18]:
trainer_stats = trainer.train()

***** Running training *****
  Num examples = 26
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 3
  Number of trainable parameters = 25,231,360


Step,Training Loss
1,2.254300
2,2.255400
3,2.130600




Training completed. Do not forget to share your model on huggingface.co/models =)




In [19]:
model.save_pretrained("lora_model")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--tinyllama-bnb-4bit/snapshots/fc56510003ea9d49362400b8a362345150802c31/config.json
Model config LlamaConfig {
  "_name_or_path": "unsloth/tinyllama",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit":

In [21]:
!pip install huggingface_hub
#model.push_to_hub("smartbrain/tiny_lora_model")

In [22]:
from huggingface_hub import notebook_login
notebook_login()

In [23]:
model.push_to_hub("smartbrain/tiny_lora_model")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--tinyllama-bnb-4bit/snapshots/fc56510003ea9d49362400b8a362345150802c31/config.json
Model config LlamaConfig {
  "_name_or_path": "unsloth/tinyllama",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit":

adapter_model.safetensors:   0%|          | 0.00/101M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/smartbrain/tiny_lora_model/commit/087fdeb1065b8d031e90cfdff0fbc13d69c64ce8', commit_message='Upload model', commit_description='', oid='087fdeb1065b8d031e90cfdff0fbc13d69c64ce8', pr_url=None, pr_revision=None, pr_num=None)